<a href="https://colab.research.google.com/github/river-li/CS521/blob/main/CS521_HW3_Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem 2

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.CIFAR10('cifar10_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.CIFAR10('cifar10_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
def tp_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return .5 * (x + delta) * (1 - ind1) * (1 - ind2) + x * ind2

def tp_smoothed_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return (x + delta) ** 2 / (4 * delta) * (1 - ind1) * (1 - ind2) + x * ind2

class Normalize(nn.Module):
    def __init__(self, mu, std):
        super(Normalize, self).__init__()
        self.mu, self.std = mu, std

    def forward(self, x):
        return (x - self.mu) / self.std

class IdentityLayer(nn.Module):
    def forward(self, inputs):
        return inputs

class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, bn, learnable_bn, stride=1, activation='relu'):
        super(PreActBlock, self).__init__()
        self.collect_preact = True
        self.activation = activation
        self.avg_preacts = []
        self.bn1 = nn.BatchNorm2d(in_planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=not learnable_bn)
        self.bn2 = nn.BatchNorm2d(planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=not learnable_bn)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=not learnable_bn)
            )

    def act_function(self, preact):
        if self.activation == 'relu':
            act = F.relu(preact)
        elif self.activation[:6] == '3prelu':
            act = tp_relu(preact, delta=float(self.activation.split('relu')[1]))
        elif self.activation[:8] == '3psmooth':
            act = tp_smoothed_relu(preact, delta=float(self.activation.split('smooth')[1]))
        else:
            assert self.activation[:8] == 'softplus'
            beta = int(self.activation.split('softplus')[1])
            act = F.softplus(preact, beta=beta)
        return act

    def forward(self, x):
        out = self.act_function(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x  # Important: using out instead of x
        out = self.conv1(out)
        out = self.conv2(self.act_function(self.bn2(out)))
        out += shortcut
        return out

class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, n_cls, cuda=True, half_prec=False,
        activation='relu', fts_before_bn=False, normal='none'):
        super(PreActResNet, self).__init__()
        self.bn = True
        self.learnable_bn = True  # doesn't matter if self.bn=False
        self.in_planes = 64
        self.avg_preact = None
        self.activation = activation
        self.fts_before_bn = fts_before_bn
        if normal == 'cifar10':
            self.mu = torch.tensor((0.4914, 0.4822, 0.4465)).view(1, 3, 1, 1)
            self.std = torch.tensor((0.2471, 0.2435, 0.2616)).view(1, 3, 1, 1)
        else:
            self.mu = torch.tensor((0.0, 0.0, 0.0)).view(1, 3, 1, 1)
            self.std = torch.tensor((1.0, 1.0, 1.0)).view(1, 3, 1, 1)
            print('no input normalization')
        if cuda:
            self.mu = self.mu.cuda()
            self.std = self.std.cuda()
        if half_prec:
            self.mu = self.mu.half()
            self.std = self.std.half()

        self.normalize = Normalize(self.mu, self.std)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=not self.learnable_bn)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.bn = nn.BatchNorm2d(512 * block.expansion)
        self.linear = nn.Linear(512*block.expansion, n_cls)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, self.bn, self.learnable_bn, stride, self.activation))
            # layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_features=False):
        for layer in [*self.layer1, *self.layer2, *self.layer3, *self.layer4]:
            layer.avg_preacts = []

        out = self.normalize(x)
        out = self.conv1(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        if return_features and self.fts_before_bn:
            return out.view(out.size(0), -1)
        out = F.relu(self.bn(out))
        if return_features:
            return out.view(out.size(0), -1)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out


def PreActResNet18(n_cls, cuda=True, half_prec=False, activation='relu', fts_before_bn=False,
    normal='none'):
    #print('initializing PA RN-18 with act {}, normal {}'.format())
    return PreActResNet(PreActBlock, [2, 2, 2, 2], n_cls=n_cls, cuda=cuda, half_prec=half_prec,
        activation=activation, fts_before_bn=fts_before_bn, normal=normal)


# intialize the model
model = PreActResNet18(10, cuda=True, activation='softplus1').to(device)
model.eval()


no input normalization


PreActResNet(
  (normalize): Normalize()
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
  )
  (layer2): Sequential(
    (0): PreActBloc

### Implement the Attacks


Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.


In [6]:
def pgd_linf_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True)
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        # TODO: compute the adv_x
        # find delta, clamp with eps
        grad_sign = adv_x.grad.data.sign()
        adv_x = adv_x.detach() + eps_step * grad_sign
        delta = torch.clamp(adv_x - x, min=-eps, max=eps)
        adv_x = torch.clamp(x + delta, min=0, max=1).detach()
    return adv_x

def pgd_l2_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True)
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        batch_size = x.size()[0]
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        grad = adv_x.grad.data
        # TODO: compute the adv_x
        # find delta, clamp with eps, project delta to the l2 ball
        # HINT: https://github.com/Harry24k/adversarial-attacks-pytorch/blob/master/torchattacks/attacks/pgdl2.py
        grad_norms = torch.norm(grad.view(batch_size, -1), p=2, dim=1) + 1e-10
        grad = grad / grad_norms.view(batch_size, 1, 1, 1)
        adv_x = adv_x.detach() + eps_step * grad
        delta = adv_x - x
        delta_norms = torch.norm(delta.view(batch_size, -1), p=2, dim=1)
        factor = eps / delta_norms
        factor = torch.min(factor, torch.ones_like(delta_norms))
        delta = delta * factor.view(-1, 1, 1, 1)

        adv_x = torch.clamp(x + delta, min=0, max=1).detach()

    return adv_x


Evaluate Single and Multi-Norm Robust Accuracy
In this section, we evaluate the model on the Linf and L2 attacks as well as union accuracy.


In [7]:
def fgsm_linf_untargeted(model, x, labels, eps):
    import torch
    import torch.nn.functional as F

    model.eval()
    x_adv = x.clone().detach().requires_grad_(True)
    logits = model(x_adv)
    loss = F.cross_entropy(logits, labels)
    grad = torch.autograd.grad(loss, x_adv)[0]

    # One FGSM step
    x_adv = x_adv + eps * torch.sign(grad)

    # Project to valid pixel range
    x_adv = torch.clamp(x_adv, 0.0, 1.0).detach()
    return x_adv

In [8]:
def test_model_on_single_attack(model, attack='pgd_linf', eps=0.1):
    model.eval()
    tot_test, tot_acc, tot_acc_adv = 0.0, 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating"):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        if attack == 'pgd_linf':
            # TODO: get x_adv untargeted pgd linf with eps, and eps_step=eps/4
            x_adv = pgd_linf_untargeted(model, x_batch, y_batch, 4, eps, eps/4)
        elif attack == 'fgsm':
            x_adv = fgsm_linf_untargeted(model, x_batch, y_batch, eps)
        else:
            pass

        # get the testing accuracy and update tot_test and tot_acc
        out_std = model(x_batch)
        pred_std = torch.max(out_std, dim=1)[1]

        out_adv = model(x_adv)
        pred_adv = torch.max(out_adv, dim=1)[1]
        tot_acc_adv += (pred_adv == y_batch).sum().item()
        tot_acc += (pred_std == y_batch).sum().item()

        tot_test += x_batch.size(0)

    print('Standard accuracy %.5lf' % (tot_acc/tot_test))
    print('Robust accuracy %.5lf' % (tot_acc_adv/tot_test), f'on {attack} attack with eps = {eps}')

## Adversarial Training

In [ ]:
# Device
import torch
import torch.nn.functional as F
from tqdm import tqdm

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = PreActResNet18(10, cuda=use_cuda, activation='softplus1', normal='cifar10').to(device)

def train_standard(model, train_loader, epochs=2, lr=0.1, weight_decay=5e-4):
    import torch.optim as optim
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay, nesterov=True)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[epochs//2, int(0.8*epochs)], gamma=0.1)
    for ep in range(1, epochs+1):
        total, correct, total_loss = 0, 0, 0.0
        pbar = tqdm(train_loader, desc=f"Standard Train Epoch {ep}/{epochs}")
        for x, y in pbar:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = F.cross_entropy(logits, y)
            loss.backward()
            optimizer.step()
            total += x.size(0)
            total_loss += loss.item() * x.size(0)
            correct += (logits.argmax(1) == y).sum().item()
            pbar.set_postfix(loss=total_loss/total, acc=correct/total)
        scheduler.step()

def train_adversarial(model, train_loader, epochs=2, lr=0.1, weight_decay=5e-4, pgd_steps=10, eps=8/255.0, eps_step=2/255.0):
    import torch.optim as optim
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay, nesterov=True)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[epochs//2, int(0.8*epochs)], gamma=0.1)
    for ep in range(1, epochs+1):
        total, correct, total_loss = 0, 0, 0.0
        pbar = tqdm(train_loader, desc=f"Adv Train Epoch {ep}/{epochs} (eps={eps:.4f})")
        for x, y in pbar:
            x, y = x.to(device), y.to(device)
            x_adv = pgd_linf_untargeted(model, x, y, k=pgd_steps, eps=eps, eps_step=eps_step)
            model.train()
            optimizer.zero_grad()
            logits = model(x_adv)
            loss = F.cross_entropy(logits, y)
            loss.backward()
            optimizer.step()
            total += x.size(0)
            total_loss += loss.item() * x.size(0)
            correct += (logits.argmax(1) == y).sum().item()
            pbar.set_postfix(loss=total_loss/total, acc=correct/total)
        scheduler.step()

def save_model(model, path):
    torch.save(model.state_dict(), path)

def new_model():
    return PreActResNet18(10, cuda=use_cuda, activation='softplus1', normal='cifar10').to(device)


# Standard training model
model_nat = new_model()
train_standard(model_nat, train_loader, epochs=10)
save_model(model_nat, f"standard_model.pth")

# Experiments for (a) and (b)
eps_values = [4/255.0, 8/255.0, 16/255.0]
for eps in eps_values:
    # Adversarial training model at this eps
    model_adv = new_model()
    train_adversarial(model_adv, train_loader, epochs=10, pgd_steps=10, eps=eps, eps_step=eps/4)
    save_model(model_adv, f"adv_pgd_eps_{int(eps*255)}_255_model.pth")


In [ ]:
def load_model(path):
    model = PreActResNet18(10, cuda=use_cuda, activation='softplus1', normal='cifar10').to(device)
    state = torch.load(path, map_location=device)
    model.load_state_dict(state)
    return model

# Evaluation: standard accuracy
@torch.no_grad()
def evaluate_standard(model, loader):
    model.eval()
    total, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        pred = logits.argmax(1)
        total += x.size(0)
        correct += (pred == y).sum().item()
    return correct / total

# Evaluation: robust accuracy under PGD
def evaluate_robust_pgd(model, loader, pgd_steps=10, eps=8/255.0, eps_step=2/255.0):
    model.eval()
    total, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        x_adv = pgd_linf_untargeted(model, x, y, k=pgd_steps, eps=eps, eps_step=eps_step)
        with torch.no_grad():
            logits = model(x_adv)
            pred = logits.argmax(1)
        total += x.size(0)
        correct += (pred == y).sum().item()
    return correct / total

# Evaluation: robust accuracy under FGSM
def evaluate_robust_fgsm(model, loader, eps=8/255.0):
    model.eval()
    total, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        x_adv = fgsm_linf_untargeted(model, x, y, eps=eps)
        with torch.no_grad():
            logits = model(x_adv)
            pred = logits.argmax(1)
        total += x.size(0)
        correct += (pred == y).sum().item()
    return correct / total

eps_values = [0.0, 8/255.0]

model_paths = [
    "standard_model.pth",
    "adv_pgd_eps_4_255_model.pth",
    "adv_pgd_eps_8_255_model.pth",
    "adv_pgd_eps_16_255_model.pth",
]

def report_model(path, eps_list, pgd_steps=20):
    model = load_model(path)
    print(f"model path: {path}")

    for eps in eps_list:
        # PGD robustness
        acc = evaluate_robust_pgd(model, test_loader, pgd_steps=pgd_steps, eps=eps, eps_step=(eps/4 if eps>0 else 0.0))

        if eps == 0.0:
            print(f"[eps=0, pgd_steps={pgd_steps:d}] Clean acc:  {acc:.4f}")
        else:
            print(f"[eps={eps:.5f}, pgd_steps={pgd_steps:d}] Robust acc: {acc:.4f}")


# Run reports
for path in model_paths:
    report_model(path, eps_values, 20)
    report_model(path, eps_values, 10)
    report_model(path, eps_values, 4)

In [9]:
# fgsm with eps = 4/255

model.load_state_dict(torch.load('standard_model.pth', map_location=device))
# Evaluate on fgsm attack with model 1 with eps = 4/255
test_model_on_single_attack(model, 'fgsm', 4/255)

model.load_state_dict(torch.load('adv_pgd_eps_4_255_model.pth', map_location=device))
# # Evaluate on fgsm attack with model 2 with eps = 4/255
test_model_on_single_attack(model, 'fgsm', 4/255)

model.load_state_dict(torch.load('adv_pgd_eps_8_255_model.pth', map_location=device))
# # Evaluate on fgsm attack with model 3 with eps = 4/255
test_model_on_single_attack(model, 'fgsm', 4/255)

model.load_state_dict(torch.load('adv_pgd_eps_16_255_model.pth', map_location=device))
# # Evaluate on fgsm attack with model 4 with eps = 4/255
test_model_on_single_attack(model, 'fgsm', 4/255)

Evaluating: 100%|██████████| 157/157 [00:12<00:00, 12.63it/s]


Standard accuracy 0.35650
Robust accuracy 0.04190 on fgsm attack with eps = 0.01568627450980392


Evaluating: 100%|██████████| 157/157 [00:10<00:00, 14.46it/s]


Standard accuracy 0.18070
Robust accuracy 0.16330 on fgsm attack with eps = 0.01568627450980392


Evaluating: 100%|██████████| 157/157 [00:10<00:00, 14.36it/s]


Standard accuracy 0.12240
Robust accuracy 0.11210 on fgsm attack with eps = 0.01568627450980392


Evaluating: 100%|██████████| 157/157 [00:11<00:00, 14.23it/s]

Standard accuracy 0.10000
Robust accuracy 0.10000 on fgsm attack with eps = 0.01568627450980392


In [10]:
# fgsm with eps = 8/255

model.load_state_dict(torch.load('standard_model.pth', map_location=device))
# Evaluate on fgsm attack with model 1 with eps = 8/255
test_model_on_single_attack(model, 'fgsm', 8/255)

model.load_state_dict(torch.load('adv_pgd_eps_4_255_model.pth', map_location=device))
# # Evaluate on fgsm attack with model 2 with eps = 8/255
test_model_on_single_attack(model, 'fgsm', 8/255)

model.load_state_dict(torch.load('adv_pgd_eps_8_255_model.pth', map_location=device))
# # Evaluate on fgsm attack with model 3 with eps = 8/255
test_model_on_single_attack(model, 'fgsm', 8/255)

model.load_state_dict(torch.load('adv_pgd_eps_16_255_model.pth', map_location=device))
# # Evaluate on fgsm attack with model 4 with eps = 8/255
test_model_on_single_attack(model, 'fgsm', 8/255)

Evaluating: 100%|██████████| 157/157 [00:11<00:00, 14.24it/s]


Standard accuracy 0.35650
Robust accuracy 0.04410 on fgsm attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [00:10<00:00, 14.32it/s]


Standard accuracy 0.18070
Robust accuracy 0.14410 on fgsm attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [00:10<00:00, 14.44it/s]


Standard accuracy 0.12240
Robust accuracy 0.10310 on fgsm attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [00:10<00:00, 14.54it/s]

Standard accuracy 0.10000
Robust accuracy 0.10000 on fgsm attack with eps = 0.03137254901960784


In [11]:
# PGD linf with eps = 4/255

model.load_state_dict(torch.load('standard_model.pth', map_location=device))
# Evaluate on Linf attack with model 1 with eps = 4/255
test_model_on_single_attack(model, 'pgd_linf', 4/255)

model.load_state_dict(torch.load('adv_pgd_eps_4_255_model.pth', map_location=device))
# # Evaluate on Linf attack with model 2 with eps = 4/255
test_model_on_single_attack(model, 'pgd_linf', 4/255)

model.load_state_dict(torch.load('adv_pgd_eps_8_255_model.pth', map_location=device))
# # Evaluate on Linf attack with model 3 with eps = 4/255
test_model_on_single_attack(model, 'pgd_linf', 4/255)

model.load_state_dict(torch.load('adv_pgd_eps_16_255_model.pth', map_location=device))
# # Evaluate on Linf attack with model 4 with eps = 4/255
test_model_on_single_attack(model, 'pgd_linf', 4/255)

Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.41it/s]


Standard accuracy 0.35650
Robust accuracy 0.01550 on pgd_linf attack with eps = 0.01568627450980392


Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.40it/s]


Standard accuracy 0.18070
Robust accuracy 0.16340 on pgd_linf attack with eps = 0.01568627450980392


Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.41it/s]


Standard accuracy 0.12240
Robust accuracy 0.11210 on pgd_linf attack with eps = 0.01568627450980392


Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.42it/s]

Standard accuracy 0.10000
Robust accuracy 0.10000 on pgd_linf attack with eps = 0.01568627450980392


In [12]:
# PGD linf with eps = 8/255

model.load_state_dict(torch.load('standard_model.pth', map_location=device))
# Evaluate on Linf attack with model 1 with eps = 8/255
test_model_on_single_attack(model, 'pgd_linf', 8/255)

model.load_state_dict(torch.load('adv_pgd_eps_4_255_model.pth', map_location=device))
# # Evaluate on Linf attack with model 2 with eps = 8/255
test_model_on_single_attack(model, 'pgd_linf', 8/255)

model.load_state_dict(torch.load('adv_pgd_eps_8_255_model.pth', map_location=device))
# # Evaluate on Linf attack with model 3 with eps = 8/255
test_model_on_single_attack(model, 'pgd_linf', 8/255)

model.load_state_dict(torch.load('adv_pgd_eps_16_255_model.pth', map_location=device))
# # Evaluate on Linf attack with model 4 with eps = 8/255
test_model_on_single_attack(model, 'pgd_linf', 8/255)

Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.40it/s]


Standard accuracy 0.35650
Robust accuracy 0.00020 on pgd_linf attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.40it/s]


Standard accuracy 0.18070
Robust accuracy 0.14390 on pgd_linf attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.41it/s]


Standard accuracy 0.12240
Robust accuracy 0.10310 on pgd_linf attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [00:35<00:00,  4.41it/s]

Standard accuracy 0.10000
Robust accuracy 0.10000 on pgd_linf attack with eps = 0.03137254901960784
